In [ ]:
# jupyter notebook
!jupyter nbextension enable --py --sys-prefix ipyleaflet

In [ ]:
##jupyter lab
!jupyter labextension install jupyter-leaflet
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python
#https://hackersandslackers.com/dealing-with-xml-in-python/

In [ ]:
pwd

#### Reading the XML Metadata files and processing Information on places

In [ ]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

In [ ]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

In [ ]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

In [ ]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

In [ ]:
identified_places = collections.defaultdict(list)
def show_record(something):
    if isinstance(something, list):
        for elem in something:
            show_record(elem)
    elif isinstance(something, collections.OrderedDict) and ('record' in something.keys()):
        try:
            #print(something['record']['Bundesland'], ' --> ', something['record']['Ort'])
            identified_places[something['record']['Bundesland']].append(something['record']['Ort'])
        except:
            #print(something['record'][0]['Bundesland'], ' --> ', something['record'][0]['Ort'])
            identified_places[something['record'][0]['Bundesland']].append(something['record'][0]['Ort'])
    elif isinstance(something, collections.OrderedDict):
        new = something['listPlace']['place']
        show_record(new)
    else:
        print(type(something))
        print(something)

In [ ]:
show_record(placeslist)

In [ ]:
identified_places.keys()

In [ ]:
df_places = pd.DataFrame(columns=['Bundesland', 'Ort'])
for key in identified_places.keys():
    df_temp = pd.DataFrame(identified_places[key], columns=['Ort'])
    df_temp['Bundesland'] = key
    df_places = pd.concat([df_places, df_temp], axis=0, sort=True)
df_places.head(30)

In [ ]:
df_places.reset_index(inplace=True, drop=True)

In [ ]:
df_places.replace('W.','Wien', inplace=True)
df_places.replace('STir.','Südtirol', inplace=True)
df_places.replace('OTir.','Osttiroler', inplace=True)
df_places.replace('NTir.','Nord tirol', inplace=True)
df_places.replace('Kä.','Kärnten', inplace=True)
df_places.replace('St.','Steiermark', inplace=True)
df_places.replace('Sa.','Salzburg', inplace=True)
df_places.replace('OÖ','Oberösterreich', inplace=True)
df_places.replace('NÖ','Niederösterreich', inplace=True)
df_places.replace('Bgl.','Burgenland', inplace=True)

In [ ]:
df_places.head(10)

In [ ]:
df_places.loc[0].values

#### Using a geolocator to get the Coordinates

In [ ]:
geolocator = gg.Nominatim(user_agent="test_application")

In [ ]:
df_places.loc[0, 'Ort'] + ', ' +  df_places.loc[0, 'Bundesland']

In [ ]:
logradouro = df_places.loc[0, 'Ort'] + ', ' +  df_places.loc[0, 'Bundesland']
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

The next cells were used just for getting geographic information. The saved Pickle shall be used for the subsequent processing. 

In [ ]:
def get_coordinates(place):
    try:
        location = geolocator.geocode(place)
    except:
        return None
    print(location)
    time.sleep(5)
    if location:
        return (location.longitude, location.latitude), location.address
    else:
        return None

In [ ]:
geoinfo = []
for idx in df_places.index:
    geoinfo.append(get_coordinates(df_places.loc[idx, 'Ort'] + ', ' +  df_places.loc[idx, 'Bundesland']))

In [ ]:
df_places['Coordinates'] = [x[0] if x else None for x in geoinfo]
df_places['Adresses'] = [x[1] if x else None for x in geoinfo]

In [ ]:
df_places.head(30)

In [ ]:
#df_places.to_pickle("./df_places.pkl")
df_places.to_excel('./df_places.xls')

In [ ]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

In [ ]:
df_places.info()

In [ ]:
df_places.Bundesland.unique()

In [ ]:
df_places[df_places.Coordinates.isnull()]

Testing with a missing field...

In [ ]:
location = geolocator.geocode('Niederösterreich')
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

In [ ]:
df_places[df_places.Coordinates.isnull()].index

In [ ]:
geoinfo_missing = []
for ort in df_places[df_places.Coordinates.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

In [ ]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Coordinates.isnull()]))

In [ ]:
idx = df_places[df_places.Coordinates.isnull()].index
df_places.loc[idx, 'Coordinates'] = [x[0] if x else None for x in geoinfo_missing]

In [ ]:
df_places[df_places.Adresses.isnull()]

In [ ]:
geoinfo_missing = []
for ort in df_places[df_places.Adresses.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

In [ ]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Adresses.isnull()]))

In [ ]:
idx = df_places[df_places.Adresses.isnull()].index
df_places.loc[idx, 'Adresses'] = [x[1] if x else None for x in geoinfo_missing]

In [ ]:
df_places.loc[df_places[df_places.Coordinates.isnull()].index]

In [ ]:
df_places.to_pickle("./df_places.pkl")

In [ ]:
#https://docs.python.org/2.4/lib/standard-encodings.html
df_places.to_csv("./df_places.csv", encoding='utf_16')

Showing places in a map

In [ ]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

In [ ]:
df_places.Coordinates[1][-1::-1]

In [ ]:
#https://ipyleaflet.readthedocs.io/en/latest/
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#https://github.com/jupyter-widgets/ipyleaflet/issues/64
#http://andrewgaidus.com/leaflet_webmaps_python/

#layer = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?", layers="nasa:bluemarble")
layer = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-30")
center = (47,14)

mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for idx in df_places.index:
    if df_places.loc[idx,'Coordinates']:
        title = df_places.loc[idx,'Ort']
        markers.append(ipyleaflet.Marker(location=df_places.loc[idx,'Coordinates'][-1::-1], draggable=False, title=title))                    
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

In [ ]:
#using Folio

mapplaces = folium.Map(location=(47,14), zoom_start=4)

mc = folium.plugins.MarkerCluster()

for idx in df_places.index:
    if df_places.loc[idx,'Coordinates']:
        title = df_places.loc[idx,'Ort']
        mc.add_child(folium.Marker(location=df_places.loc[idx,'Coordinates'][-1::-1], popup=title))
                       
mapplaces.add_child(mc)
mapplaces

In [ ]:
mapplaces.save('./places.html')

#### Reading the XML Data files:

In [ ]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [ ]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [ ]:
root = tree.getroot()
print(root)

In [ ]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

In [ ]:
for child in root:
    print(child.tag, child.attrib)

In [ ]:
set([elem.tag for elem in root.iter()])

In [ ]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = collections.Counter(places)
freq_places.most_common(10)

In [ ]:
for elem in root.iter('{http://www.tei-c.org/ns/1.0}entry'):
    print(elem.attrib)
    #print(elem.attrib['type'])
    #print(elem.get('entry'))
    #print(elem.find('orth type').text)

In [ ]:
def show(elem, indent = 0):
    print(' ' * indent + elem.tag)
    for child in elem.findall('*'):
         show(elem, indent + 1)
            
show(root)

In [ ]:
for elem1 in root.iter('{http://www.tei-c.org/ns/1.0}entry'):
    print(elem.findall('orth'))
    #print(elem.attrib['type'])
    #print(elem.get('entry'))
    #print(elem.find('orth type').text)

#for beer in e.findall('beer'):
#    name = beer.get('name') # equivalent to .attrib() in this case
#    flavor = beer.find('flavor').text
#    print(name, " is ", flavor)

Dictionary Approach

In [ ]:
xml2dictfile = xmltodict.parse((open(xmlfile, mode='r', encoding='utf-8').read()))

In [ ]:
print(xml2dictfile['TEI'].keys())

In [ ]:
xml2dictfile['TEI']['@xmlns']
#xml2dictfile['TEI']['teiHeader']

In [ ]:
print(len(xml2dictfile['TEI']['text']['body']['entry']))
xml2dictfile['TEI']['text']['body']['entry'][0].keys()

In [ ]:
xml2dictfile['TEI']['text']['body']['entry'][144]

In [ ]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

In [ ]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('{}{}/{}'.format(datapath, folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles):
    print(i+1, '--' ,w)

In [ ]:
df_lemmas = pd.DataFrame(columns=['Ort', 'Lemma'])
orts = []
lemmas = []

for idx1, filex in enumerate(onlyfiles):
    xmldictfile = xmltodict.parse((open(filex, encoding="utf-8").read()))
    for idx2, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
        idx = ((idx1+1)*(idx2+1)-1)
        
        ort = None
        if 'usg' in entry.keys():
            if isinstance(entry['usg'], list):
                for local in entry['usg']:
                    try:
                        ort = local['placeName']['#text']
                    except:
                        ort = local['placeName']
            else:
                try:
                    ort = entry['usg']['placeName']['#text']
                except:
                    ort = entry['usg']['placeName']
        orts.append(ort)

        lemma = None
        if 'form' in entry.keys():
            if isinstance(entry['form'], list):
                for elem in entry['form']:
                    if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                        lemma = elem['orth']
                    elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                        for subelem in elem['orth']:
                            if subelem['@type'] == 'normalized':
                                lemma = subelem['#text']
            else:
                if isinstance(entry['form']['orth'], list):
                    for elem in entry['form']['orth']:
                        if elem['@type'] == 'normalized':
                            lemma = elem['#text']
                else:
                    lemma = entry['form']['orth']
        lemmas.append(lemma)
        
        if idx % 10000 == 0:
            print(idx)

df_lemmas['Ort'] = orts
df_lemmas['Lemma'] = lemmas

In [ ]:
df_lemmas.info()

In [ ]:
df_lemmas.head(40)

In [ ]:
df_lemmas = pd.read_pickle("./df_lemmas.pkl")
df_lemmas.head(100)

In [ ]:
grouped = df_lemmas.groupby('Ort')['Lemma'].apply(list)

In [ ]:
grouped.head()

In [ ]:
len(grouped)

#### Georeferencing Lemmas

In [ ]:
lemma_places = []
for p in df_freqdist.Place:
    lemma_places.append(get_coordinates(p))
    
lemma_places

In [ ]:
idx = df_freqdist.Place.index
idx

In [ ]:
with open('./df_freqdist.pkl', 'rb') as f:
    df_freqdist = pickle.load(f)

df_freqdist.head()

In [ ]:
with open('./df_joined.pkl', 'rb') as f:
    df_joined = pickle.load(f)

df_joined.head(10)

In [ ]:
df_joined.reset_index(inplace=True)

In [ ]:
df_joined.head(10)

In [ ]:
df_joined.to_csv('./df_joined.csv')

In [ ]:
def list2string(lista):
    texto = ''
    for elem in lista:
        texto = texto + '\n' + elem[0] + ' --- ' + str(elem[1]) + '\n'
    return texto

In [ ]:
df_joined.Lemma

In [ ]:
center = (47,14)
mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for idx in df_joined.index:
    if df_joined.loc[idx,'Coordinates'] and not isinstance(df_joined.loc[idx,'Lemma'], float):
        title = list2string(collections.Counter(df_joined.loc[idx,'Lemma']).most_common(6))
        markers.append(ipyleaflet.Marker(location=df_joined.loc[idx,'Coordinates'][-1::-1], draggable=False, title=title))
                       
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

In [ ]:
#using Folio

mapplaces = folium.Map(location=(47,14),
               zoom_start=4)

mc = folium.plugins.MarkerCluster()

for idx in df_joined.index:
     if df_joined.loc[idx,'Coordinates'] and not isinstance(df_joined.loc[idx,'Lemma'], float):
        title = list2string(collections.Counter(df_joined.loc[idx,'Lemma']).most_common(6))
        mc.add_child(folium.Marker(location=df_joined.loc[idx,'Coordinates'][-1::-1], popup=title))
        
                     
mapplaces.add_child(mc)
mapplaces

In [ ]:
mapplaces.save('./lemmas_places.html')